# Travaux pratiques 4

Le but de ce TP est de manipuler les différentes notions abordées dans la vidéo du module 4, consacrée à la prédiction. Pour cela, nous utiliserons un jeu de données réel sans valeurs manquantes, auquel nous ajouterons des valeurs manquantes synthétiques pour contrôler leur effet.

Rappelons le contexte de la prédiciton : à partir de nouvelles données $X_{\mathrm{new}}$, nous voulons prédire une variable cible $y_{\mathrm{new}}$. Pour cela, nous apprenons un modèle sur un jeu de données d'entraînement $(X_{\mathrm{train}},y_{\mathrm{train}})$, où nous connaissons la variable cible.

Le TP comporte trois exercices.

* Dans l'exercice 1, nous établirons des résultats de référence sans valeurs manquantes, afin de pouvoir évaluer l'effet de celles-ci dans la suite.
* Dans l'exercice 2, nous considérerons le cas où seul le jeu d'entraînement est incomplet. L'objectif est de manipuler les méthodes d'imputation déjà vues dans le TP2 pour entraîner un modèle sur un jeu de données $X_{\mathrm{train}}$ incomplet.
* Dans l'exercice 3, nous étudierons le cas où il y a des valeurs manquantes à la fois dans le jeu d'entraînement, et dans les nouvelles données sur lesquelles nous cherchons à prédire la réponse. L'objectif est de comparer les stratégies *one-step* et *two-step* lorsque $X_{\mathrm{new}}$ est également incomplet.

# Introduction

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

## Librairies importées dans la solution

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.ensemble import RandomForestRegressor

## Importation et préparation des données

Tout au long du TP, vous utiliserez le jeu de données classique *California Housing Prices* (https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html) avec le preprocessing proposé par Scikit-Learn pour n'avoir que des variables numériques.

Dans un deuxième temps, vous pouvez refaire le TP en repartant du jeu de données brut pour améliorer les résultats. En effet, en pratique, le choix d'encodage des variables non numériques peut impacter la gestion des valeurs manquantes.

In [ ]:
X_full, y = fetch_california_housing(return_X_y=True, as_frame=False)

Comme d'habitude en apprentissage supervisé, on veut étudier la capacité de généralisation d'un modèle sur de nouvelles données $X_{\mathrm{new}}$.

En fait, on peut différencier trois jeux de données : le jeu d'entraînement $(X_{\mathrm{train}},y_{\mathrm{train}})$ sur lequel le modèle est appris, le jeu de test $(X_{\mathrm{test}},y_{\mathrm{test}})$ sur lequel on valide le modèle et pour lequel on a accès à la variable cible (ce sont un peu des nouvelles données factices), et les nouvelles données $X_{\mathrm{new}}$ sur lequel le modèle est appliqué et pour lequel on veut prédire la variable cible.

En pratique, on a souvent deux jeux disjoints par construction, par exemple lorsqu'on entraîne un modèle sur des données d'archive (jeu qui sera découpé en un jeu d'entraînement et en un jeu de test), et qu'on veut le tester sur les données qui viennent d'être collectées (nouvelles données).

Ici, nous allons séparer le jeu d'entraînement et de test aléatoirement.

In [ ]:
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, test_size=0.2)

# Exercice 1 : régression de référence sur le jeu de données complet

Tout l'intérêt d'utiliser un dataset sans `NAs` natifs est de pouvoir établir un score de référence. On établira un tel score sur le jeu de test, puis on le validera statistiquement par validation croisée.

## Question 1 : prédiction

Choisissez un modèle de régression que vous utiliserez dans tout le TP (par exemple en utilisant `sklearn.ensemble`). Entraînez-le sur `X_train_full`, puis évaluez son erreur quadratique moyenne (MSE) sur `X_test_full`.

### Solution

On peut choisir une forêt aléatoire comme modèle de régression avec `RandomForestRegressor`.

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(X_train_full, y_train)

y_pred = regressor.predict(X_test_full)
mse_test_full = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse_test_full:.4f}")

## Question 2 : validation croisée

Afin d'établir un intervalle de confiance sur ce score de généralisation, réalisez une validation croisée du modèle sur `X_train`. Le score sur le jeu de test est-il dans l'intervalle de confiance ?

### Solution

In [ ]:
regressor = RandomForestRegressor()

mse_cv_full = - cross_val_score(
    regressor,
    X_train_full, y_train,
    cv=10, scoring='neg_mean_squared_error')

In [ ]:
mu = mse_cv_full.mean()
sigma = mse_cv_full.std()
print(f"CV MSE: {mu:.4f} +/- {2*sigma:.4f} --- [{mu-2*sigma:.4f} ; {mu+2*sigma:.4f}] (95% confidence)")
print(f"Test MSE: {mse_test_full:.4f}")

Le score sur le jeu de test est bien dans l'intervalle de confiance. Cela nous rassure sur l'absence de dérive de distribution entre le train et le test (même si ici on le sait déjà puisqu'on a coupé train et test aléatoirement, mais en pratique, ce n'est pas forcément le cas).

# Exercice 2 : Seul le jeu d'entraînement est incomplet

## Question 1 : génération de valeurs manquantes

En reprenant du code des TPs précédents, générez des valeurs manquantes de type MCAR (lorsque le fait qu'il y a des valeurs manquantes est totalement indépendant des valeurs des données elles-mêmes) sur une copie de `X_train_full`, que vous pourrez appeler `X_train_miss`, avec une probabilité de manque de $p=0.5$ sur toutes les variables.

### Solution

In [ ]:
X_train_miss = np.copy(X_train_full)

p = 0.5
n, d = X_train_miss.shape

for j in range(d):
  miss_id = (np.random.uniform(0, 1, size=n) < p)
  X_train_miss[miss_id, j] = np.nan

## Question 2 : imputation par la moyenne

En utilisant le module `sklearn.impute`, imputez `X_train_miss` par ses moyennes. Vous appelerez le résultat `X_train_imp`.

### Solution

In [ ]:
imputer = SimpleImputer(strategy='mean')
X_train_imp = imputer.fit_transform(X_train_miss)

## Question 3 : prédiction

Entraînez le même modèle de régression que dans l'exercice 1 sur `X_train_imp`, puis évaluez sa MSE sur `X_test_full`. Comparez l'erreur avec le cas sans valeurs manquants.

### Solution

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(X_train_imp, y_train)

y_pred = regressor.predict(X_test_full)
mse_test_imp = mean_squared_error(y_test, y_pred)
print(f"Test MSE (mean imputation): {mse_test_imp:.4f}")

La MSE est supérieure au cas sans valeurs manquantes, ce qui est normal puisque nous avons moins d'information.

## Question 4 : cas complet

Comparez le résultat avec celui obtenu en supprimant toutes les lignes incomplètes. Combien reste-t-il de lignes ?

### Solution

In [ ]:
X_train_cc = X_train_miss[~np.isnan(X_train_miss).any(axis=1)]
y_train_cc = y_train[~np.isnan(X_train_miss).any(axis=1)]

print(f"Initialement: {X_train_miss.shape[0]} rows.")
print(f"Complete case: {X_train_cc.shape[0]} rows.")

regressor = RandomForestRegressor()
regressor.fit(X_train_cc, y_train_cc)

y_pred = regressor.predict(X_test_full)
mse_test_cc = mean_squared_error(y_test, y_pred)
print(f"Test MSE (complete case): {mse_test_cc:.4f}")

En supprimant toutes les lignes incomplètes, on n'a plus que 66 lignes sur 16512 ! Logiquement, la MSE est largement supérieure.

## Question 5 : imputation itérative

Comparez le résultat avec une imputation plus élaborée, en utilisant par exemple la classe `IterativeImputer` de Scikit-learn. Peut-on conclure sur l'intérêt de cette imputation plus élaborée ?

### Solution

In [ ]:
imputer = IterativeImputer()
X_train_ii = imputer.fit_transform(X_train_miss)

regressor = RandomForestRegressor()
regressor.fit(X_train_ii, y_train)

y_pred = regressor.predict(X_test_full)
mse_test_ii = mean_squared_error(y_test, y_pred)
print(f"Test MSE (iterative imputation): {mse_test_ii:.4f}")

Les scores sont proches. Sans intervalle de confiance, il est difficile de conclure sur l'apport de l'imputation itérative par rapport à la moyenne dans ce cas.

## Question 6 : le piège de la validation de modèle

Nous allons à présent tenter d'appliquer la même stratégie de validation croisée que dans l'exercice 1, pour avoir un intervalle de confiance sur les MSE et comparer les méthodes d'imputation. Repartez du jeu d'entraînement imputé par la moyenne `X_train_imp`, et faites une cross-validation de votre modèle dessus. Le score du jeu de test est-il dans l'intervalle de confiance ? Que peut-on en conclure ?

### Solution

In [ ]:
regressor = RandomForestRegressor()

mse_cv_imp = - cross_val_score(
    regressor,
    X_train_imp, y_train,
    cv=10, scoring='neg_mean_squared_error')

In [ ]:
mu = mse_cv_imp.mean()
sigma = mse_cv_imp.std()
print(f"CV MSE: {mu:.4f} +/- {2*sigma:.4f} --- [{mu-2*sigma:.4f} ; {mu+2*sigma:.4f}] (95% confidence)")
print(f"Test MSE: {mse_test_imp:.4f}")

Le résultat sur le jeu de test est très différent (ici bien meilleur) que dans la cross-validation. Cela est dû à une dérive de distribution entre train et test. En introduisant des valeurs manquantes dans `X_train` et pas dans `X_test`, on se place dans un cas non i.i.d., ce qui empêche de valider le modèle par cross-validation. On ne pourra donc pas aller plus loin.

Cette situation arrive souvent en pratique. Par exemple, dix ans de données d'archive sont disponibles pour entraîner un modèle mais avec des problèmes de valeurs manquantes (entre autres), et un nouveau protocole a été mis en place depuis quelques semaines pour enregistrer des données propres et valider le modèle dessus.

# Exercice 3 : Le jeu d'entraînement et les nouvelles données sont incomplètes

A présent, nous considérons qu'il y a des valeurs manquantes dans `X_test` également, et qu'il n'y a donc pas de dérive de distribution. On se ramène donc à un cas i.i.d.

## Question 1 : génération de valeurs manquantes

Comme dans l'exercice 2, générez des valeurs manquantes sur une copie de `X_test_full`, que vous pourrez appeler `X_test_miss`.

### Solution

In [ ]:
X_test_miss = np.copy(X_test_full)

p = 0.5
n, d = X_test_miss.shape

for j in range(d):
  miss_id = (np.random.uniform(0, 1, size=n) < p)
  X_test_miss[miss_id, j] = np.nan

## Question 2 : stratégie two-step

Implémentez un pipeline avec une imputation qui fonctionne *out-of-sample* et une régression (vous pourrez utiliser les modules `sklearn.pipeline` et `sklearn.impute`). Entraînez le pipeline sur `X_train_miss` et réaliser une prédiction sur `X_test_miss`. Comparez la MSE obtenue avec celle sans valeurs manquantes.

### Solution

In [ ]:
pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('regressor', RandomForestRegressor())
])

pipeline.fit(X_train_miss, y_train)

y_pred = pipeline.predict(X_test_miss)
mse_test_mistest = mean_squared_error(y_test, y_pred)
print(f"Test MSE (NA in test): {mse_test_mistest:.4f}")

Comme attendu, la MSE est nettement supérieure au cas sans valeurs manquantes.

## Question 3 : validation croisée

Réalisez une validation croisée sur `X_train_miss` et vérifiez que le score sur `X_test_miss` est dans l'intervalle de confiance de la validation croisée.


### Solution

In [ ]:
pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('regressor', RandomForestRegressor())
])

mse_cv_mistest = - cross_val_score(
    pipeline,
    X_train_miss, y_train,
    cv=10, scoring='neg_mean_squared_error')

In [ ]:
mu = mse_cv_mistest.mean()
sigma = mse_cv_mistest.std()
print(f"CV MSE: {mu:.4f} +/- {2*sigma:.4f} --- [{mu-2*sigma:.4f} ; {mu+2*sigma:.4f}] (95% confidence)")
print(f"Test MSE: {mse_test_mistest:.4f}")

## Question 4 : stratégie one-step

Entraînez sur `X_train_miss` un modèle de régression à base d'arbres fonctionnant sur données incomplètes avec la technique one-step (utilisez `sklearn.ensemble`). Réalisez une prédiction sur `X_test_miss`. Comparez la MSE obtenue avec la stratégie two-steps.

### Solution

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train_miss, y_train)

y_pred = regressor.predict(X_test_miss)
mse_test_1s = mean_squared_error(y_test, y_pred)
print(f"Test MSE (one step): {mse_test_1s:.4f}")

## Question 5 : validation croisée

Précisez ce résultat avec une validation croisée.

### Solution

In [ ]:
regressor = RandomForestRegressor()

mse_cv_1s = - cross_val_score(
    regressor,
    X_train_miss, y_train,
    cv=10, scoring='neg_mean_squared_error')

In [ ]:
mu = mse_cv_1s.mean()
sigma = mse_cv_1s.std()
print(f"CV MSE: {mu:.4f} +/- {2*sigma:.4f} --- [{mu-2*sigma:.4f} ; {mu+2*sigma:.4f}] (95% confidence)")
print(f"Test MSE: {mse_test_1s:.4f}")